In [ ]:
import numpy as np
import random
import matplotlib.patheffects as PathEffects

import tensorflow as tf
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

from tensorflow.keras.models import Model, load_model, Sequential
import efficientnet.tfkeras as efn 
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import regularizers
from tweaked_ImageGenerator_v2 import ImageDataGenerator as SeqImageGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical 
import preprocess_crop
import tensorflow as tf
import math
from tensorflow.keras.callbacks import Callback
from random_eraser import get_random_eraser

In [ ]:
height, width = 299,100
batch_size=32
nb_frame = 1
input_image_shape = (nb_frame,height,width,3)
train_dir = 'MARKET1501/market_rename/train_all'
epochs = 120
learning_rate_base = 3e-08

# Data Preprocessing

In [ ]:

# The data, split between train and test sets
train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    color_mode="rgb",
    batch_size=50000,
    class_mode='binary',
    shuffle=True,
    subset='training'
)
#datagen_val = ImageDataGenerator()

# validation_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(height, width),
#     color_mode="rgb",
#     batch_size=5000,
#     class_mode='binary',
#     shuffle=True,
#     subset='validation'
# )
x_train1, y_train1 = train_generator.next()
# x_val1, y_val1 = validation_generator.next()
x_train1 = x_train1.astype('float32')
# x_val11 = x_val1.astype('float32')
y_train1 = y_train1.astype('float32')
# y_val1 = y_val1.astype('float32')


In [ ]:
y_train1

In [ ]:
class ReshapeLayer(Layer):
    def call(self,inputs):
        nshape = (12936,1) + inputs.shape[1:]
        return tf.reshape(inputs,nshape)

In [ ]:
datagen = SeqImageGenerator(horizontal_flip=True,rotation_range=10,shear_range=0.2,
                            preprocessing_function=get_random_eraser(v_l=0, v_h=255))
# datagen.fit(x_train1)
seq_gen=datagen.flow_from_directory(train_dir, target_size=(height, width), batch_size=batch_size, frames_per_step=nb_frame)

steps_per_epoch=12936//batch_size
print(steps_per_epoch)
# datagenval = ImageDataGenerator()
# val_data=datagenval.flow_from_directory(valid_folder, target_size=(height, width), batch_size=batch_size, frames_per_step=nb_frame)

# Triplet Center Loss

In [ ]:

def triplet_center_loss(y_true, y_pred, n_classes= 10, alpha=0.38):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ', y_pred)

    total_lenght = y_pred.shape.as_list()[-1]
    print('total_lenght=',  total_lenght)
    #     total_lenght =12

    # repeat y_true for n_classes and == np.arange(n_classes)
    # repeat also y_pred and apply mask
    # obtain min for each column min vector for each class

    classes = tf.range(0, n_classes,dtype=tf.float32)
    y_pred_r = tf.reshape(y_pred, (tf.shape(y_pred)[0], 1))
    y_pred_r = tf.keras.backend.repeat(y_pred_r, n_classes)

    y_true_r = tf.reshape(y_true, (tf.shape(y_true)[0], 1))
    y_true_r = tf.keras.backend.repeat(y_true_r, n_classes)

    mask = tf.equal(y_true_r[:, :, 0], classes)

    #mask2 = tf.ones((tf.shape(y_true_r)[0], tf.shape(y_true_r)[1]))  # todo inf

    # use tf.where(tf.equal(masked, 0.0), np.inf*tf.ones_like(masked), masked)

    masked = y_pred_r[:, :, 0] * tf.cast(mask, tf.float32) #+ (mask2 * tf.cast(tf.logical_not(mask), tf.float32))*tf.constant(float(2**10))
    masked = tf.where(tf.equal(masked, 0.0), np.inf*tf.ones_like(masked), masked)

    minimums = tf.math.reduce_min(masked, axis=1)

    loss = K.max(y_pred - minimums +alpha ,0)

    # obtain a mask for each pred


    return loss

# Attention Block

In [ ]:
def spatial_attention(input_feature):
    #kernel_size = 7
    kernel_size = 3
    
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)	
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])



# Model

In [ ]:
from keras_self_attention import SeqSelfAttention

def fusion_3(image_input_shape,embedding_size,n_class,height,width):
   
    y = Input(shape=(n_class,))
    input_image = Input(batch_shape=(None, seq_len,height, width, 3))
    eff_model=efn.EfficientNetB3(input_shape=(height, width, 3),
                                 include_top=False,
                                 weights='noisy-student')
    model_backbone = Model(eff_model.input,eff_model.get_layer('block7a_project_bn').output)
    timeDistributed_layer = tf.keras.layers.TimeDistributed(model_backbone)(input_image)
    print("TimeDistributed", timeDistributed_layer.shape)
    
    '''Temporal'''
    t = tf.keras.layers.TimeDistributed(GlobalAveragePooling2D())(timeDistributed_layer)
    t = LSTM(256, return_sequences=True, input_shape=(t.shape[1],t.shape[2]), name="lstm_layer_in")(t)
    t = SeqSelfAttention(attention_activation='sigmoid')(t)
    avg_pool = GlobalAveragePooling1D()(t)
    max_pool = GlobalMaxPooling1D()(t)
    t = concatenate([avg_pool, max_pool])
    
    t = Dropout(0.3)(t)
    print("Temporal: ", t.shape)
    
    '''Spatial'''
    s = tf.math.reduce_mean(timeDistributed_layer, axis=1)   
    s = SeparableConv2D(filters = 512, kernel_size = (3, 3), padding = 'same')(s)
    s = spatial_attention(s)
    s = SeparableConv2D(filters = 512, kernel_size = (3, 3), padding = 'same')(s)
    s = spatial_attention(s)
    s = BatchNormalization()(s)
    a = GlobalAveragePooling2D()(s)
    c = Dropout(0.3)(a)
    print("Spatial: ", s.shape)
    
        
    '''Fusion'''
    f = tf.keras.layers.Concatenate()([c, t])
    f = Dropout(0.3)(f)
    print("Fusion: ", f.shape)
    return f,y,input_image  

def fc_reid(x,y,n_class):
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x =  BatchNormalization()(x)
    softmax = Dense(n_class, activation='softmax', name='reid_output')(x)

    center = Embedding(n_class, embedding_size)(y)
    l2_loss = Lambda(lambda x: K.sum(K.square(x[0] - x[1][:, 0]), 1, keepdims=True), name='l2_loss')(
        [x, center])
    return softmax,l2_loss

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr                    


# Create & Compile

In [ ]:
def create_model_fusion(image_input_shape,embedding_size,n_class,height,width):
     
    loss_weights = [1, 0.0005]
    model,y,input_image = fusion_3(image_input_shape,embedding_size,n_class,height,width)
    softmax_id ,l2_loss = fc_reid(model,y,n_class)
    optimizer = tfa.optimizers.LazyAdam(learning_rate_base)
    lr_metric = get_lr_metric(optimizer)
    model = tf.keras.models.Model(inputs=[input_image,y], outputs=[softmax_id, l2_loss]) 
    model.compile(loss={'reid_output':tf.keras.losses.CategoricalCrossentropy(), 
                        'l2_loss':triplet_center_loss},
                  optimizer=optimizer,metrics=['accuracy',lr_metric],
                  loss_weights={'reid_output':1,
                               'l2_loss':0.0005}
                 )
    model.summary()
    return model

In [ ]:
seq_len =1
n_class = 751
embedding_size =128
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1", "/gpu:2"], cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
# with mirrored_strategy.scope():
model = create_model_fusion(input_image_shape,embedding_size,n_class,height,width)   

# START TRAINING

In [ ]:
filepath = 'model_two_stream'

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='reid_output_loss',
                             verbose=1,
                             save_best_only=True)

# stop = EarlyStopping(monitor='val_loss', patience =5,
#                       verbose=0, mode='auto', baseline=None, 
#                       restore_best_weights=False)
def scheduler(epoch, lr =learning_rate_base):
    if epoch <= 20:
#         return 0.0003
#         return lr * math.exp(0.45)
        return lr * 1.59
    if epoch > 20 and epoch <=70:  
        return 0.0005
    if epoch > 70 and epoch <=100:
        return 0.00005
    if epoch > 100:
        return 0.000005
    
def scheduler_tuned(epoch, lr =learning_rate_base):
    if epoch <= 2:
        return 0.0000003
my_lr_tuned = tf.keras.callbacks.LearningRateScheduler(scheduler_tuned)

my_lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks = [checkpoint,my_lr_callback]
# callbacks = [checkpoint, my_lr_tuned]

history = model.fit_generator(
    seq_gen,
#     validation_data=(gen_val),
    verbose=1,
    shuffle=True,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,     
#     validation_steps=steps_per_epoch_val,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['accuracy'],
                    name='Train'))
# fig.add_trace(go.Scatter(
#                     y=history.history['loss'],
#                     name='Loss'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for Re-identification feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

plt.clf()
fig = go.Figure()

fig.add_trace(go.Scatter(
                    y=history.history['loss'],
                    name='Loss'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for Re-identification feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

# Save and load last weights

In [ ]:
# model.save('combine.hdf5')

In [ ]:
model.load_weights("model_combine_fix/variables/variables")
